Import modules

In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
import importlib
from sklearn import metrics
import pickle
import Yifeng_task
import learning_dash
import opal_dash
import pandas as pd

import itertools

%matplotlib inline


Initialize task

In [2]:
dynaPRL = Yifeng_task.DynaPRL(minblock_len=25, maxblock_len=40, block_rand=0.1, n_trials=1000, p_high=[0.45, 0.6, 0.8, 0.8], p_low=[0.45, 0.3, 0.1, 0.2], reward_manipulate=0)

Set up parameters

In [7]:

#variable parameters

alpha_a_Go_values=np.arange(0.1, 1.1, 0.2)
alpha_a_NoGo_values=np.arange(0.1, 1.1, 0.2)
alpha_c_values = [0.01, 0.05, 0.1]
beta_GO_values = [1, 1.5, 2, 2.5, 3]
beta_NOGO_values = [1, 1.5, 2, 2.5, 3]

num_var_parameters=[alpha_a_Go_values, alpha_a_NoGo_values, alpha_c_values, beta_GO_values, beta_NOGO_values]
params_combos=list(itertools.product(*num_var_parameters))

# w=input('Save warmup trials? Enter: Y/N')

# if w=='Y':
#     save_warmup=True
# elif w=='N':
#     save_warmup=False
# else:
#     print('Please enter Y or N.')
#     save_warmup=False

save_warmup = False


In [8]:
import concurrent.futures
from simu_utils import gridsimu_wrapper  # Import the wrapper function
from tqdm import tqdm

# Prepare arguments for the wrapper
args_list = [(params, dynaPRL, save_warmup) for params in params_combos]

# Function to handle the progress bar
def tqdm_map(func, iterable, **kwargs):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(func, arg) for arg in iterable]
        results = []
        for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), **kwargs):
            results.append(f.result())
        return results
    
# Execute in parallel
if __name__ == '__main__':
    # Execute in parallel
    # with concurrent.futures.ProcessPoolExecutor() as executor:
    #     results = list(executor.map(gridsimu_wrapper, args_list))

    results = tqdm_map(gridsimu_wrapper, args_list, desc="Processing Simulations")

    # Concatenate results into a single DataFrame
    GridSimu_Opal_dynaPRL_all = pd.DataFrame()
    for data in results:
        GridSimu_Opal_dynaPRL_all = pd.concat([GridSimu_Opal_dynaPRL_all, data], ignore_index=True)

Processing Simulations: 100%|██████████| 1875/1875 [11:01<00:00,  2.83it/s]


In [9]:
GridSimu_Opal_dynaPRL_all.to_csv('GridSimu_Opal_dynaPRL_all_1k1s_6b.csv', index=False)

In [10]:
print(len(GridSimu_Opal_dynaPRL_all))

1781250
